# GB BESS Market Exploratory Analysis

This notebook loads the collected data from both the **Elexon Insights Solution API** and the **NESO Data Portal**, then explores:

1. **DC/DR/DM Auction Results** — clearing prices and volumes by service type
2. **System Prices** — SSP/SBP half-hourly trends and spreads
3. **Market Index** — imbalance price distributions
4. **Generation Mix** — fuel type composition over time
5. **Cross-source analysis** — system price vs. DC clearing price correlation

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

RAW = Path("../data/raw")

# Display settings
pd.set_option("display.max_columns", 40)
pd.set_option("display.float_format", "{:.2f}".format)

## 1. Load Data

In [ ]:
# NESO: DC/DR/DM auction results (Jan–Oct 2023)
auctions = pd.read_csv(RAW / "auction_results_2023-01-01_2023-10-31.csv", parse_dates=["EFA Date", "Delivery Start", "Delivery End"])

# Elexon: system prices, market index, generation (Jul–Oct 2023)
sys_prices = pd.read_csv(RAW / "system_prices_2023-07-01_2023-10-31.csv", parse_dates=["settlementDate", "startTime"])
mkt_index  = pd.read_csv(RAW / "market_index_2023-07-01_2023-10-31.csv", parse_dates=["settlementDate", "startTime"])
gen_fuel   = pd.read_csv(RAW / "generation_by_fuel_2023-07-01_2023-10-31.csv", parse_dates=["settlementDate", "startTime", "publishTime"])

print(f"Auctions:      {auctions.shape}")
print(f"System prices: {sys_prices.shape}")
print(f"Market index:  {mkt_index.shape}")
print(f"Gen by fuel:   {gen_fuel.shape}")

## 2. DC/DR/DM Auction Results

In [ ]:
auctions.head()

In [ ]:
# Summary stats per service
auction_stats = auctions.groupby("Service").agg(
    avg_clearing_price=("Clearing Price", "mean"),
    median_clearing_price=("Clearing Price", "median"),
    max_clearing_price=("Clearing Price", "max"),
    avg_volume=("Cleared Volume", "mean"),
    total_records=("Clearing Price", "count"),
).round(2)

auction_stats

In [ ]:
# Clearing price time series by service
fig = px.line(
    auctions,
    x="EFA Date",
    y="Clearing Price",
    color="Service",
    title="DC/DR/DM Clearing Prices Over Time (2023)",
    labels={"Clearing Price": "Clearing Price (£/MW/h)", "EFA Date": "Date"},
)
fig.update_layout(height=500, template="plotly_white")
fig.show()

In [ ]:
# Box plot: price distributions by service
fig = px.box(
    auctions,
    x="Service",
    y="Clearing Price",
    color="Service",
    title="Clearing Price Distribution by Service",
    labels={"Clearing Price": "£/MW/h"},
)
fig.update_layout(height=450, template="plotly_white", showlegend=False)
fig.show()

In [ ]:
# Weekly average clearing prices by service
auctions["week"] = auctions["EFA Date"].dt.to_period("W").dt.start_time
weekly = auctions.groupby(["week", "Service"])["Clearing Price"].mean().reset_index()

fig = px.line(
    weekly,
    x="week",
    y="Clearing Price",
    color="Service",
    title="Weekly Average Clearing Prices by Service",
    labels={"Clearing Price": "£/MW/h", "week": "Week"},
)
fig.update_layout(height=500, template="plotly_white")
fig.show()

In [ ]:
# Clearing price by EFA block (time-of-day pattern)
fig = px.box(
    auctions,
    x="EFA",
    y="Clearing Price",
    color="Service",
    title="Clearing Price by EFA Block (Intraday Pattern)",
    labels={"Clearing Price": "£/MW/h", "EFA": "EFA Block (1=23:00–03:00 ... 6=19:00–23:00)"},
)
fig.update_layout(height=500, template="plotly_white")
fig.show()

## 3. System Prices (SSP / SBP)

In [ ]:
sys_prices.head()

In [ ]:
# Daily average system prices
daily_sp = sys_prices.groupby("settlementDate").agg(
    avg_sell_price=("systemSellPrice", "mean"),
    avg_buy_price=("systemBuyPrice", "mean"),
    max_sell_price=("systemSellPrice", "max"),
    min_sell_price=("systemSellPrice", "min"),
).reset_index()

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.08,
                    subplot_titles=("Daily Average System Prices", "Daily Price Range (SSP)"))

fig.add_trace(go.Scatter(x=daily_sp["settlementDate"], y=daily_sp["avg_sell_price"],
                         name="Avg SSP", line=dict(color="#EF553B")), row=1, col=1)
fig.add_trace(go.Scatter(x=daily_sp["settlementDate"], y=daily_sp["avg_buy_price"],
                         name="Avg SBP", line=dict(color="#636EFA")), row=1, col=1)

fig.add_trace(go.Scatter(x=daily_sp["settlementDate"], y=daily_sp["max_sell_price"],
                         name="Max SSP", line=dict(color="#EF553B", dash="dot")), row=2, col=1)
fig.add_trace(go.Scatter(x=daily_sp["settlementDate"], y=daily_sp["min_sell_price"],
                         name="Min SSP", line=dict(color="#00CC96", dash="dot")), row=2, col=1)

fig.update_layout(height=600, template="plotly_white", title_text="System Sell & Buy Prices (Jul–Oct 2023)")
fig.update_yaxes(title_text="£/MWh", row=1, col=1)
fig.update_yaxes(title_text="£/MWh", row=2, col=1)
fig.show()

In [ ]:
# Price spread (SBP - SSP) — when these diverge, imbalance costs are high
sys_prices["spread"] = sys_prices["systemBuyPrice"] - sys_prices["systemSellPrice"]

fig = px.histogram(
    sys_prices,
    x="spread",
    nbins=80,
    title="Distribution of SBP – SSP Spread",
    labels={"spread": "£/MWh (Buy – Sell)"},
)
fig.update_layout(height=400, template="plotly_white")
fig.show()

print(f"Spread stats:")
print(sys_prices["spread"].describe().round(2))

In [ ]:
# System price by settlement period (intraday profile)
sp_by_period = sys_prices.groupby("settlementPeriod")["systemSellPrice"].agg(["mean", "median", "std"]).reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=sp_by_period["settlementPeriod"], y=sp_by_period["mean"],
                         name="Mean", mode="lines+markers"))
fig.add_trace(go.Scatter(x=sp_by_period["settlementPeriod"], y=sp_by_period["median"],
                         name="Median", mode="lines", line=dict(dash="dash")))

fig.update_layout(
    title="System Sell Price by Settlement Period (Intraday Profile)",
    xaxis_title="Settlement Period (1=00:00, 48=23:30)",
    yaxis_title="£/MWh",
    height=450, template="plotly_white",
)
fig.show()

## 4. Market Index Prices

In [ ]:
# Filter to the primary data provider (APXMIDP) — the other is often zero-volume
mkt_primary = mkt_index[mkt_index["dataProvider"] == "APXMIDP"].copy()

daily_mkt = mkt_primary.groupby("settlementDate").agg(
    avg_price=("price", "mean"),
    total_volume=("volume", "sum"),
).reset_index()

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.08,
                    subplot_titles=("Daily Average Market Index Price", "Daily Total Volume"))

fig.add_trace(go.Scatter(x=daily_mkt["settlementDate"], y=daily_mkt["avg_price"],
                         name="Avg Price", line=dict(color="#AB63FA")), row=1, col=1)
fig.add_trace(go.Bar(x=daily_mkt["settlementDate"], y=daily_mkt["total_volume"],
                     name="Volume", marker_color="#FFA15A"), row=2, col=1)

fig.update_layout(height=550, template="plotly_white", title_text="Market Index Price & Volume")
fig.update_yaxes(title_text="£/MWh", row=1, col=1)
fig.update_yaxes(title_text="MWh", row=2, col=1)
fig.show()

## 5. Generation Mix

In [ ]:
# Daily generation by fuel type
daily_gen = gen_fuel.groupby(["settlementDate", "fuelType"])["generation"].sum().reset_index()

# Pivot for stacked area chart
gen_pivot = daily_gen.pivot_table(index="settlementDate", columns="fuelType", values="generation", fill_value=0)

# Order by total generation (descending)
col_order = gen_pivot.sum().sort_values(ascending=False).index
gen_pivot = gen_pivot[col_order]

fig = px.area(
    gen_pivot.reset_index().melt(id_vars="settlementDate", var_name="Fuel Type", value_name="Generation"),
    x="settlementDate",
    y="Generation",
    color="Fuel Type",
    title="Daily Generation by Fuel Type (Jul–Oct 2023)",
    labels={"Generation": "MW", "settlementDate": "Date"},
)
fig.update_layout(height=550, template="plotly_white")
fig.show()

In [ ]:
# Average share by fuel type
fuel_share = gen_pivot.mean()
fuel_share = fuel_share[fuel_share > 0].sort_values(ascending=False)

fig = px.pie(
    values=fuel_share.values,
    names=fuel_share.index,
    title="Average Generation Share by Fuel Type",
)
fig.update_layout(height=450)
fig.show()

## 6. Cross-Source Analysis: System Price vs DC Clearing Price

In [ ]:
# Prepare daily averages from both sources for the overlapping period (Jul–Oct 2023)
daily_sys = sys_prices.groupby("settlementDate")["systemSellPrice"].mean().reset_index()
daily_sys.columns = ["date", "avg_system_price"]

# DC High clearing prices (most relevant for BESS)
dc_high = auctions[auctions["Service"] == "DCH"].copy()
daily_dc = dc_high.groupby("EFA Date")["Clearing Price"].mean().reset_index()
daily_dc.columns = ["date", "avg_dc_clearing_price"]

# Merge on overlapping dates
merged = pd.merge(daily_sys, daily_dc, on="date", how="inner")
print(f"Overlapping days: {len(merged)}")

if not merged.empty:
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=merged["date"], y=merged["avg_system_price"],
                             name="Avg System Price (SSP)", line=dict(color="#EF553B")),
                  secondary_y=False)
    fig.add_trace(go.Scatter(x=merged["date"], y=merged["avg_dc_clearing_price"],
                             name="Avg DC High Clearing Price", line=dict(color="#636EFA")),
                  secondary_y=True)

    fig.update_layout(title="System Price vs DC High Clearing Price",
                      height=500, template="plotly_white")
    fig.update_yaxes(title_text="System Price (£/MWh)", secondary_y=False)
    fig.update_yaxes(title_text="DC Clearing Price (£/MW/h)", secondary_y=True)
    fig.show()

In [ ]:
# Correlation
if not merged.empty:
    corr = merged[["avg_system_price", "avg_dc_clearing_price"]].corr()
    print(f"Pearson correlation: {corr.iloc[0, 1]:.3f}")

    fig = px.scatter(
        merged,
        x="avg_system_price",
        y="avg_dc_clearing_price",
        trendline="ols",
        title=f"System Price vs DC Clearing Price (r = {corr.iloc[0, 1]:.3f})",
        labels={"avg_system_price": "Avg SSP (£/MWh)", "avg_dc_clearing_price": "Avg DC High (£/MW/h)"},
    )
    fig.update_layout(height=450, template="plotly_white")
    fig.show()

## 7. Summary Statistics

In [ ]:
print("=" * 60)
print("DATASET SUMMARY")
print("=" * 60)

print(f"\nNESO Auction Results: {len(auctions):,} records")
print(f"  Date range: {auctions['EFA Date'].min().date()} to {auctions['EFA Date'].max().date()}")
print(f"  Services: {sorted(auctions['Service'].unique())}")

print(f"\nElexon System Prices: {len(sys_prices):,} records")
print(f"  Date range: {sys_prices['settlementDate'].min().date()} to {sys_prices['settlementDate'].max().date()}")
print(f"  SSP range: £{sys_prices['systemSellPrice'].min():.2f} – £{sys_prices['systemSellPrice'].max():.2f}/MWh")

print(f"\nElexon Market Index: {len(mkt_index):,} records")
print(f"  Data providers: {sorted(mkt_index['dataProvider'].unique())}")

print(f"\nElexon Generation by Fuel: {len(gen_fuel):,} records")
print(f"  Fuel types: {sorted(gen_fuel['fuelType'].unique())}")